In [1]:
import malmo
import malmo.MalmoPython as MalmoPython
import os
import sys
import time
import json
import numpy as np

import gym
from gym import error, spaces, utils
from gym.utils import seeding

In [2]:
import functools
print = functools.partial(print, flush=True)

In [3]:
def Menger(xorg, yorg, zorg, size, blocktype, variant, holetype):
    #draw solid chunk
    genstring = GenCuboidWithVariant(xorg,yorg,zorg,xorg+size-1,yorg+size-1,zorg+size-1,blocktype,variant) + "\n"
    #now remove holes
    unit = size
    while (unit >= 3):
        w=unit//3
        for i in range(0, size, unit):
            for j in range(0, size, unit):
                x=xorg+i
                y=yorg+j
                genstring += GenCuboid(x+w,y+w,zorg,(x+2*w)-1,(y+2*w)-1,zorg+size-1,holetype) + "\n"
                y=yorg+i
                z=zorg+j
                genstring += GenCuboid(xorg,y+w,z+w,xorg+size-1, (y+2*w)-1,(z+2*w)-1,holetype) + "\n"
                genstring += GenCuboid(x+w,yorg,z+w,(x+2*w)-1,yorg+size-1,(z+2*w)-1,holetype) + "\n"
        unit = w
    return genstring

def GenCuboid(x1, y1, z1, x2, y2, z2, blocktype):
    return '<DrawCuboid x1="' + str(x1) + '" y1="' + str(y1) + '" z1="' + str(z1) + '" x2="' + str(x2) + '" y2="' + str(y2) + '" z2="' + str(z2) + '" type="' + blocktype + '"/>'

def GenCuboidWithVariant(x1, y1, z1, x2, y2, z2, blocktype, variant):
    return '<DrawCuboid x1="' + str(x1) + '" y1="' + str(y1) + '" z1="' + str(z1) + '" x2="' + str(x2) + '" y2="' + str(y2) + '" z2="' + str(z2) + '" type="' + blocktype + '" variant="' + variant + '"/>'

In [4]:
drawingDecorator = '''<DrawingDecorator>
                    <DrawSphere x="-27" y="70" z="0" radius="30" type="air"/>''' + Menger(-40, 40, -13, 27, "stone", "smooth_granite", "air") + '''
                    <DrawCuboid x1="-25" y1="39" z1="-2" x2="-29" y2="39" z2="2" type="lava"/>
                    <DrawCuboid x1="-26" y1="39" z1="-1" x2="-28" y2="39" z2="1" type="obsidian"/>
                    <DrawBlock x="-27" y="39" z="0" type="diamond_block"/>
                  </DrawingDecorator>'''

missionXML='''<?xml version="1.0" encoding="UTF-8" standalone="no" ?>
            <Mission xmlns="http://ProjectMalmo.microsoft.com" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance">
            
              <About>
                <Summary>Hello world!</Summary>
              </About>
              
            <ServerSection>
              <ServerInitialConditions>
                <Time>
                    <StartTime>1000</StartTime>
                    <AllowPassageOfTime>false</AllowPassageOfTime>
                </Time>
                <Weather>clear</Weather>
              </ServerInitialConditions>
              <ServerHandlers>
                  <FlatWorldGenerator generatorString="3;7,44*49,73,35:1,159:4,95:13,35:13,159:11,95:10,159:14,159:6,35:6,95:6;12;"/>
                  <DrawingDecorator>
                      <DrawBlock x="0" y="56" z="-1" type="diamond_block" />
                  </DrawingDecorator>
                  <ServerQuitFromTimeUp timeLimitMs="10000"/>
                  <ServerQuitWhenAnyAgentFinishes/>
                </ServerHandlers>
              </ServerSection>
              
              <AgentSection mode="Survival">
                <Name>MalmoTutorialBot</Name>
                <AgentStart>
                    <Placement x="0.5" y="56.0" z="0.5" yaw="180"/>
                </AgentStart>
                <AgentHandlers>
                  <ObservationFromFullStats/>
                  <ObservationFromGrid>
                      <Grid name="floor3x3">
                        <min x="-1" y="-1" z="-1"/>
                        <max x="1" y="0" z="1"/>
                      </Grid>
                  </ObservationFromGrid>
                  <ObservationFromDistance>
                      <Marker name="Origin" x="0.5" y="56.0" z="0.5" />
                  </ObservationFromDistance>
                  <ContinuousMovementCommands turnSpeedDegs="180"/>
                </AgentHandlers>
              </AgentSection>
            </Mission>'''




In [5]:
def setup_malmo() -> [MalmoPython.AgentHost, MalmoPython.MissionSpec, MalmoPython.MissionRecordSpec]:
    # Create default Malmo objects:

    agent_host = MalmoPython.AgentHost()
    try:
        agent_host.parse( sys.argv )
    except RuntimeError as e:
        print('ERROR:',e)
        print(agent_host.getUsage())
        exit(1)
    if agent_host.receivedArgument("help"):
        print(agent_host.getUsage())
        exit(0)

    my_mission = MalmoPython.MissionSpec(missionXML, True)
    my_mission_record = MalmoPython.MissionRecordSpec()

    # Attempt to start a mission:
    max_retries = 3
    for retry in range(max_retries):
        try:
            agent_host.startMission( my_mission, my_mission_record )
            break
        except RuntimeError as e:
            if retry == max_retries - 1:
                print("Error starting mission:",e)
                exit(1)
            else:
                time.sleep(2)

    # Loop until mission starts:
    print("Waiting for the mission to start ", end=' ')
    world_state = agent_host.getWorldState()
    while not world_state.has_mission_begun:
        print(".", end="")
        time.sleep(0.1)
        world_state = agent_host.getWorldState()
        for error in world_state.errors:
            print("Error:",error.text)

    print()
    print("Mission running ", end=' ')


    return agent_host, my_mission, my_mission_record

In [6]:
def try_observation(agent_host):
    world_state = agent_host.getWorldState()
    for error in world_state.errors:
            print("Error:",error.text)
            
    finished = not world_state.is_mission_running
    state = None
    
    if world_state.number_of_observations_since_last_state > 0:
        msg = world_state.observations[-1].text                 # Yes, so get the text
        observations = json.loads(msg)                          # and parse the JSON
        grid = observations.get(u'floor3x3', 0)                 # and get the grid we asked for
        #dist = observations.get(u'distanceFromOrigin', 0)
        #px = observations.get(u'XPos', 0)
        #py = observations.get(u'YPos', 0)
        #print(px, py, pz)
        dist = 0.5 - observations.get(u'ZPos', 0)
        
        state = grid, dist
    
    return finished, state

In [7]:
def get_observation(agent_host):
    done, state = try_observation(agent_host)
    
    while(not done and state is None):
        time.sleep(0.1)
        done, state = try_observation(agent_host)
    
    grid, dist = None, None
    if(state is not None):
        grid, dist = state
    return done, grid, dist

In [8]:
def convert_observation(malmo_obs, output_shape):
    '''
    Returns the observation in the form of a MultiBinary space of the right shape
    1 if block is occupied, 0 if block is empty
    '''
    array = np.array(malmo_obs).reshape(output_shape)
    return np.where(array == 'air', 0, 1)

In [20]:
def handle_action(agent_host, actions):
    #print(actions)
    mov, jmp = actions
    mov -= 1 # between -1 and 1
    
    agent_host.sendCommand("move " + str(mov))
    agent_host.sendCommand("jump " + str(jmp))
    time.sleep(0.1)
    agent_host.sendCommand("jump 0")
    agent_host.sendCommand("move 0")

In [21]:
class MinecraftEnv(gym.Env):
    
    metadata = {'render.modes': ['human']}

    def __init__(self):        
        # set up environment variables (state space, action space, rewards...)
        self.obs_shape = (2 * 3 * 3,) #[y, z, x]
        self.observation_space = spaces.MultiBinary(self.obs_shape[0])
        self.action_space = spaces.MultiDiscrete((3, 2)) # [0] => move -1 / 0 / 1 ; [1] => jump 0 / 1
        self.distance = 0
    
    def step(self, action):
        # handle action
        handle_action(self.agent_host, action)
        # get new observation
        finished, grid, dist = get_observation(self.agent_host)
        if finished:
            return None, 0, True, {}
        
        obs = convert_observation(grid, self.obs_shape)
        # calculate reward
        reward = dist - self.distance
        self.distance = dist
        
        return obs, reward, finished, {}
    
    def reset(self):
        # set up malmo
        agent_host, my_mission, my_mission_record = setup_malmo()
        self.agent_host = agent_host
        self.my_mission = my_mission
        self.my_mission_record = my_mission_record
        self.distance = 0
        
        finished, grid, dist = get_observation(self.agent_host)
        return convert_observation(grid, self.obs_shape)
    
    def render(self, mode='human'):
        pass
  
    def close(self):
        pass

In [12]:
################ Test random agent

env = MinecraftEnv()
for i_episode in range(20):
    observation = env.reset()
    for t in range(100):
        env.render()
        #print(observation)        
        action = env.action_space.sample()
        observation, reward, done, info = env.step(action)
        print(reward)
        if done:
            print("Episode finished after {} timesteps".format(t+1))
            break
env.close()


ERROR: Caught std::exception: unrecognised option '-f'

Malmo version: 0.36.0

Allowed options:
  -h [ --help ]         show description of allowed options
  --test                run this as an integration test


Waiting for the mission to start  ..
Mission running  0.0
0.06880688675142321
0.105247351825007
0.07547460573056458
0.07991687692845684
0.11983897388809206
0.25037319124023866
0.2811489256678983
0.19325387765576818
0.16684649239109284
0.14532611429039366
0.12094707585125097
0.11881700895267766
0.18220900888154556
0.1255930786766366
0.10704834983031031
-0.08828452870820591
-0.0036695464155216406
0.06818003783138726
0.11638144690195151
0.11471478768317755
0.05453624101219923
-0.03791910316007918
0.11075792975689325
0.19798921540340286
0.14178741509462212
0.10705854499155176
0.07952650812377282
0.06415997195051792
0.048813134656331414
0.039047107211700016
0.004939211452697823
0.0005736921448260546
0.033785887721303975
0.0191708364847214
0.008021956797202634
0.00854070494721082
0

-0.07184980842090694
-0.1730546479622439
-0.14304417033808026
-0.12637343294833503
-0.12043970757856837
-0.11188382940618258
0.03515004245542652
0.062125112813035344
0.0685778772204797
0.06968271345258703
0.04654124123268488
0.04728957250972776
0.1599478417135698
0.15575238787359424
0.1738173580989657
0.2077221273169152
0.11454023496865995
0.3510740519509925
0.3946114250350763
0.3388268585495656
0.25715733960011455
0.18232518846501122
0.1172702888032714
0.047442635829353874
-0.004898131026893127
0.00012059331449265631
0.005308076506480575
0.002536080952268094
-0.01115863025188446
-0.017621968498716445
0.011181286723849104
0.01457189647544127
-0.015334431106484303
-0.05256838929097818
-0.13366040823066377
-0.10797959925270967
-0.04823762979748508
-0.07639683443819573
-0.025406715065369512
-0.003226891692748257
0.01275097634368949
0.001550509026583935
ERROR: Caught std::exception: unrecognised option '-f'

Malmo version: 0.36.0

Allowed options:
  -h [ --help ]         show description o

-0.0014091977121801769
0.0900386841633476
0.0961130805964916
0.061305988187907556
0.023313171461766924
0.01553216857871903
-0.07094083595141432
-0.075896220002513
-0.022967168924118597
0.03592228760252636
0.031014505193045938
0.13010932927532975
0.06758703578642539
0.015913135445272564
-0.045169533531162154
-0.11011107463957398
-0.13615409621246022
-0.13435760610520492
-0.12664175751783657
0.004145800178457115
0.10427149402299951
0.16121923109352115
0.20837765487981547
0.21752865599980564
0.27063906542192284
0.11115761974874439
0.030316290403331836
-0.018473406863709685
0.015923882461635452
0.16212540433320677
0.27962071471953753
0.11773589551400354
0.23777528725415076
0.25350458775190177
0.2624509915707227
0.2614660317034563
0.2800424456896726
0.1497547170211906
0.3817919268694485
0.2835311858763756
0.20416537182524408
0.09865575254591619
0.021775279072798526
-0.0030062637495218425
0.052075304870784045
0.03940582372060053
-0.17888548117228265
-0.18523718074937
-0.05057414938052407
-0.

KeyboardInterrupt: 

In [ ]:
######################x With real baseline algorithms

from stable_baselines.common.policies import MlpPolicy
from stable_baselines.common import make_vec_env, env_checker
from stable_baselines.common.vec_env import DummyVecEnv
from stable_baselines import A2C
from stable_baselines.common.vec_env.util import copy_obs_dict, dict_to_obs, obs_space_info

# Parallel environments
#env_checker.check_env(MinecraftEnv())
#env = make_vec_env(MinecraftEnv, n_envs=1)
env = MinecraftEnv()

model = A2C(MlpPolicy, env, verbose=1)
model.learn(total_timesteps=25000)
model.save("a2c_cartpole")

del model # remove to demonstrate saving and loading

model = A2C.load("a2c_cartpole")

env = gym.make('CartPole-v1')
obs = env.reset()
while True:
    time.sleep(0.1)
    action, _states = model.predict(obs)
    obs, rewards, dones, info = env.step(action)
    env.render()


Wrapping the env in a DummyVecEnv.
ERROR: Caught std::exception: unrecognised option '-f'

Malmo version: 0.36.0

Allowed options:
  -h [ --help ]         show description of allowed options
  --test                run this as an integration test


Waiting for the mission to start  ....
Mission running  ---------------------------------
| explained_variance | 0.513    |
| fps                | 5        |
| nupdates           | 1        |
| policy_entropy     | 1.79     |
| total_timesteps    | 5        |
| value_loss         | 0.554    |
---------------------------------
Error: AgentHost::sendCommand : commands connection is not open. Is the mission running?
Error: AgentHost::sendCommand : commands connection is not open. Is the mission running?
ERROR: Caught std::exception: unrecognised option '-f'

Malmo version: 0.36.0

Allowed options:
  -h [ --help ]         show description of allowed options
  --test                run this as an integration test


Waiting for the mission to star

ERROR: Caught std::exception: unrecognised option '-f'

Malmo version: 0.36.0

Allowed options:
  -h [ --help ]         show description of allowed options
  --test                run this as an integration test


Waiting for the mission to start  .....
Mission running  Error: AgentHost::sendCommand : commands connection is not open. Is the mission running?
Error: AgentHost::sendCommand : commands connection is not open. Is the mission running?
ERROR: Caught std::exception: unrecognised option '-f'

Malmo version: 0.36.0

Allowed options:
  -h [ --help ]         show description of allowed options
  --test                run this as an integration test


Waiting for the mission to start  ....
Mission running  Error: AgentHost::sendCommand : commands connection is not open. Is the mission running?
Error: AgentHost::sendCommand : commands connection is not open. Is the mission running?
ERROR: Caught std::exception: unrecognised option '-f'

Malmo version: 0.36.0

Allowed options:
  -h [ 

Waiting for the mission to start  .....
Mission running  Error: AgentHost::sendCommand : commands connection is not open. Is the mission running?
Error: AgentHost::sendCommand : commands connection is not open. Is the mission running?
ERROR: Caught std::exception: unrecognised option '-f'

Malmo version: 0.36.0

Allowed options:
  -h [ --help ]         show description of allowed options
  --test                run this as an integration test


Waiting for the mission to start  ....
Mission running  Error: AgentHost::sendCommand : commands connection is not open. Is the mission running?
Error: AgentHost::sendCommand : commands connection is not open. Is the mission running?
ERROR: Caught std::exception: unrecognised option '-f'

Malmo version: 0.36.0

Allowed options:
  -h [ --help ]         show description of allowed options
  --test                run this as an integration test


Waiting for the mission to start  .....
Mission running  ----------------------------------
| explained

ERROR: Caught std::exception: unrecognised option '-f'

Malmo version: 0.36.0

Allowed options:
  -h [ --help ]         show description of allowed options
  --test                run this as an integration test


Waiting for the mission to start  ....
Mission running  ---------------------------------
| explained_variance | -1.92    |
| fps                | 8        |
| nupdates           | 900      |
| policy_entropy     | 1.57     |
| total_timesteps    | 4500     |
| value_loss         | 0.419    |
---------------------------------
Error: AgentHost::sendCommand : commands connection is not open. Is the mission running?
Error: AgentHost::sendCommand : commands connection is not open. Is the mission running?
ERROR: Caught std::exception: unrecognised option '-f'

Malmo version: 0.36.0

Allowed options:
  -h [ --help ]         show description of allowed options
  --test                run this as an integration test


Waiting for the mission to start  .....
Mission running  Error: Ag

Waiting for the mission to start  .....
Mission running  Error: AgentHost::sendCommand : commands connection is not open. Is the mission running?
Error: AgentHost::sendCommand : commands connection is not open. Is the mission running?
ERROR: Caught std::exception: unrecognised option '-f'

Malmo version: 0.36.0

Allowed options:
  -h [ --help ]         show description of allowed options
  --test                run this as an integration test


Waiting for the mission to start  ....
Mission running  Error: AgentHost::sendCommand : commands connection is not open. Is the mission running?
Error: AgentHost::sendCommand : commands connection is not open. Is the mission running?
ERROR: Caught std::exception: unrecognised option '-f'

Malmo version: 0.36.0

Allowed options:
  -h [ --help ]         show description of allowed options
  --test                run this as an integration test


Waiting for the mission to start  ....
Mission running  Error: AgentHost::sendCommand : commands connec

Waiting for the mission to start  ....
Mission running  Error: AgentHost::sendCommand : commands connection is not open. Is the mission running?
Error: AgentHost::sendCommand : commands connection is not open. Is the mission running?
ERROR: Caught std::exception: unrecognised option '-f'

Malmo version: 0.36.0

Allowed options:
  -h [ --help ]         show description of allowed options
  --test                run this as an integration test


Waiting for the mission to start  .....
Mission running  Error: AgentHost::sendCommand : commands connection is not open. Is the mission running?
Error: AgentHost::sendCommand : commands connection is not open. Is the mission running?
ERROR: Caught std::exception: unrecognised option '-f'

Malmo version: 0.36.0

Allowed options:
  -h [ --help ]         show description of allowed options
  --test                run this as an integration test


Waiting for the mission to start  .....
Mission running  Error: AgentHost::sendCommand : commands conne

ERROR: Caught std::exception: unrecognised option '-f'

Malmo version: 0.36.0

Allowed options:
  -h [ --help ]         show description of allowed options
  --test                run this as an integration test


Waiting for the mission to start  ....
Mission running  Error: AgentHost::sendCommand : commands connection is not open. Is the mission running?
Error: AgentHost::sendCommand : commands connection is not open. Is the mission running?
ERROR: Caught std::exception: unrecognised option '-f'

Malmo version: 0.36.0

Allowed options:
  -h [ --help ]         show description of allowed options
  --test                run this as an integration test


Waiting for the mission to start  ....
Mission running  ---------------------------------
| explained_variance | -1.18    |
| fps                | 8        |
| nupdates           | 1900     |
| policy_entropy     | 1.42     |
| total_timesteps    | 9500     |
| value_loss         | 5.96     |
---------------------------------
Error: Age